In [23]:
import pandas as pd
import re
import html
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb

In [24]:
NER_COLS = ["ORG", "GPE", "NORP", "DATE", "CARDINAL", "PRODUCT", "ORDINAL", "LOC", "LAW"]
BASE_COLS = ["keyword", "country"]
LABEL_COL = "PCL_category"

In [25]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, nlp, stop_words):
        self.nlp = nlp
        self.ner_cols = NER_COLS
        self.base_cols = BASE_COLS
        self.stop_words = stop_words
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        
        texts = X["text"].astype(str).tolist()
        results = []
        
        for i, doc in enumerate(self.nlp.pipe(texts, batch_size=32)):
 
            row_features = {
                # Constructed features
                **self._clean_text(doc),
                **self._punctuation_features(doc),
                **self._ner_counts(doc),
                **self._verb_noun_ratio(doc),
                # Base columns
                "keyword": X.iloc[i]["keyword"],
                "country": X.iloc[i]["country"]
            }
            
            
            results.append(row_features)
        
        return pd.DataFrame(results)
    
    def _clean_text(self, doc):
        text = doc.text
        text = html.unescape(text)
        text = re.sub(r"\s+", " ", text.replace("\n", " ").replace("\t", " ")).strip()
        text = re.sub(r"[^\w\s]", "", text)
        words = text.lower().split()
        tokens = [w for w in words if w not in self.stop_words]
        return {"cleaned_text": " ".join(tokens)}

    def _punctuation_features(self, doc):
        text = doc.text
        sentence_len = max(len(text.split()), 1)
        pct_exclam = text.count("!") / sentence_len
        pct_question = text.count("?") / sentence_len
        return {"pct_exclam": pct_exclam, "pct_question": pct_question}

    def _ner_counts(self, doc):
        counts = dict.fromkeys(self.ner_cols, 0)
        for ent in doc.ents:
            if ent.label_ in counts:
                counts[ent.label_] += 1
        return counts
    
    def _verb_noun_ratio(self, doc):
        num_verbs = sum(1 for token in doc if token.pos_ == "VERB")
        num_nouns = sum(1 for token in doc if token.pos_ == "NOUN")
        
        ratio = num_verbs / num_nouns if num_nouns > 0 else 0
        return {"verb_noun_ratio": ratio}

In [26]:
def data_preprocess(data_path: str):
    df = pd.read_csv(
        data_path,
        sep="\t",
        skiprows=9,
        engine="python",
        index_col=0,
        header=None,
        names = ["article_id", "keyword", "country", "text", "PCL_category"]
    )
    y_categorical = df[LABEL_COL]
    y_binary = (y_categorical >= 2).astype(int)
    X = df.drop(columns=[LABEL_COL])
    X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [27]:
categorical_cols = ["keyword", "country"] 
numerical_cols =  NER_COLS + ["verb_noun_ratio", "pct_exclam", "pct_question"] 
cleaned_txt_col = "cleaned_text"

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [29]:
data_path = "dontpatronizeme_pcl.tsv"
X_train, X_test, y_train, y_test = data_preprocess(data_path)

neg = np.sum(y_train == 0)
pos = np.sum(y_train == 1)
scale_pos_weight = neg / pos

nlp = spacy.load("en_core_web_sm")
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

feature_extractor = FeatureExtractor(nlp, stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/berra.dogan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("tfidf", TfidfVectorizer(max_features=5000), cleaned_txt_col),
        ("numeric", StandardScaler(), numerical_cols),
    ],
    remainder="drop"
)

pipeline = Pipeline([
    ("features", feature_extractor),
    ("preprocess", preprocessor),
    ("model", xgb.XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        random_state=42
    ))
])


In [31]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureExtractor(nlp=<spacy.lang.en.English object at 0x309543250>,
                                  stop_words={'a', 'about', 'above', 'after',
                                              'again', 'against', 'ain', 'all',
                                              'am', 'an', 'and', 'any', 'are',
                                              'aren', "aren't", 'as', 'at',
                                              'be', 'because', 'been', 'before',
                                              'being', 'below', 'between',
                                              'both', 'but', 'by', 'can',
                                              'couldn', "couldn't", ...})),
                ('preprocess',
                 ColumnTransf...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [ ]:
from sklearn.metrics import f1_score, classification_report
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

print(classification_report(y_test, y_pred))


F1 score: 0.36585365853658536
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      2835
           1       0.31      0.44      0.37       305

    accuracy                           0.85      3140
   macro avg       0.62      0.67      0.64      3140
weighted avg       0.88      0.85      0.86      3140

